<h1 align="center">LAB1S1_p4. Contextual word-embeddings for text representation</h1>

<h3 style="display:block; margin-top:5px;" align="center">Natural Language and Information Retrieval</h3>
<h3 style="display:block; margin-top:5px;" align="center">Degree in Data Science</h3>
<h3 style="display:block; margin-top:5px;" align="center">2024-2025</h3>    
<h3 style="display:block; margin-top:5px;" align="center">ETSInf. Universitat Politècnica de València</h3>
<br>

In [9]:
!pip install -U transformers
!pip install -U emoji
!pip install -U ipywidgets

/usr/bin/pip:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import load_entry_point
Traceback (most recent call last):
  File "/usr/bin/pip", line 11, in <module>
    load_entry_point('pip==20.0.2', 'console_scripts', 'pip')()
  File "/home/zzzdream/.local/lib/python3.8/site-packages/pkg_resources/__init__.py", line 542, in load_entry_point
    return get_distribution(dist).load_entry_point(group, name)
  File "/home/zzzdream/.local/lib/python3.8/site-packages/pkg_resources/__init__.py", line 3219, in load_entry_point
    return ep.load()
  File "/home/zzzdream/.local/lib/python3.8/site-packages/pkg_resources/__init__.py", line 2781, in load
    return self.resolve()
  File "/home/zzzdream/.local/lib/python3.8/site-packages/pkg_resources/__init__.py", line 2787, in resolve
    module = __import__(self.module_name, fromlist=['__name__'], level=0)
  File "/usr/lib/python3/dist-packages/pip/_in

## Some libraries

In [ ]:
import pandas as pd
import torch
from transformers import AutoModel, AutoTokenizer
from transformers import BertTokenizer, BertModel, RobertaTokenizer, RobertaModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

2025-04-10 13:23:47.927187: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-10 13:23:47.980793: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-10 13:23:48.449199: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-10 13:23:50.364467: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Read the corpora

In [ ]:
filepath = {
    "english": "EXIST2024_EN_examples_mini.csv",
    "spanish": "EXIST2024_ES_examples_mini.csv"
}
df = {k: pd.read_csv(v, sep="\t") for k, v in filepath.items()}

en_data = df["english"]
es_data = df["spanish"]

# OR USING Google colab
#from google.colab import drive
#drive.mount('/content/drive')
#df = {
#    "english": pd.read_csv("/content/drive/MyDrive/LNR/LNR2025/Lab1/EXIST2024_EN_examples.csv", sep="\t"),
#    "spanish": pd.read_csv("/content/drive/MyDrive/LNR/LNR2025/Lab1/EXIST2024_ES_examples.csv", sep="\t")
#}


## Model names

In [ ]:
modelnames = {
    "english": ["bert-base-uncased", "roberta-base"],
    "spanish": ["dccuchile/bert-base-spanish-wwm-uncased", "PlanTL-GOB-ES/roberta-base-bne"]
}

## Which device to use?

In [ ]:
if torch.backends.mps.is_available():  # Mac M? GPU
    device = torch.device("mps")
elif torch.cuda.is_available():  # Nvidia GPU
    device = torch.device("cuda")
else:  # CPU
    device = torch.device("cpu")
print(device)

cpu


## Load the tokenizers and the models

In [ ]:
# COMPLETE
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

bert_spanish_tokenizer = BertTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")
roberta_spanish_tokenizer = RobertaTokenizer.from_pretrained("PlanTL-GOB-ES/roberta-base-bne")

model_bert = BertModel.from_pretrained("bert-base-uncased")
model_roberta = RobertaModel.from_pretrained('roberta-base')

model_bert_spanish = BertModel.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")
model_roberta_spanish = RobertaModel.from_pretrained("PlanTL-GOB-ES/roberta-base-bne")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at PlanTL-GOB-ES/roberta-base-bne and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Compute tweets representations

In [ ]:
def get_embeddings(texts, model, tokenizer, batch_size=16):
    model.to(device).eval()

    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        encoded = tokenizer(batch, padding=True, truncation=True, max_length=512, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model(**encoded)
            encoded_layers = outputs[0]
            cls_vector = encoded_layers[:,0,:]

        embeddings.append(cls_vector)

    cls_vector = torch.cat(embeddings).cpu()
    return cls_vector

In [ ]:
en_bert = get_embeddings(en_data["text"].tolist(), model_bert, bert_tokenizer)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
en_roberta = get_embeddings(en_data["text"].tolist(), model_roberta, roberta_tokenizer)

In [ ]:
es_bert = get_embeddings(es_data["text"].tolist(), model_bert_spanish, bert_spanish_tokenizer)

In [ ]:
es_roberta = get_embeddings(es_data["text"].tolist(), model_roberta_spanish, roberta_spanish_tokenizer)

In [ ]:
en_labels = en_data["label"].values
es_labels = es_data["label"].values

## Compute cosine similarities

In [ ]:
# COMPLETE
def find_most_similar_pairs(embeddings, labels, texts, category):
    indices = np.where(labels == category)[0]
    sub_embeddings = embeddings[indices]
    similarity_matrix = cosine_similarity(sub_embeddings)
    np.fill_diagonal(similarity_matrix, -1)
    max_idx = np.unravel_index(np.argmax(similarity_matrix), similarity_matrix.shape)
    max_similarity = similarity_matrix[max_idx].round(4)
    return texts[indices[max_idx[0]]], texts[indices[max_idx[1]]], max_similarity

In [ ]:
def show_results(model, labels, data, model_name):
    sentence1_no, sentence2_no, similarity_no = find_most_similar_pairs(model, labels, data, "NO")
    sentence1_yes, sentence2_yes, similarity_yes = find_most_similar_pairs(model, labels, data, "YES")

    print(model_name)
    print("-------------------\n")

    print("Label: NO")
    print("sentence1:", sentence1_no)
    print("--------------------")
    print("sentence2:", sentence2_no)
    print("distance:", similarity_no, "\n\n")


    print("Label: YES")
    print("sentence1:", sentence1_yes)
    print("--------------------")
    print("sentence2:", sentence2_yes)
    print("distance:", similarity_yes, "\n\n")

In [ ]:
show_results(en_bert, en_labels, en_data["text"].tolist(), "bert-base-uncased")
show_results(en_roberta, en_labels, en_data["text"].tolist(), "roberta-base")
show_results(es_bert, es_labels, es_data["text"].tolist(), "dccuchile/bert-base-spanish-wwm-uncased")
show_results(es_roberta, es_labels, es_data["text"].tolist(), "PlanTL-GOB-ES/roberta-base-bne")

bert-base-uncased
-------------------

Label: NO
sentence1: I still wish they turned this into a boss fight. https://t.co/HyvPYJPHJc
--------------------
sentence2: I don't particularly care or want to know about the cock carousel. Everyone has a past. https://t.co/73WMTyEKHt
distance: 0.9739 


Label: YES
sentence1: The mighty ass. Call me sexist I do not care. https://t.co/LzXw4iRbLR
--------------------
sentence2: @RP_JetBlack Not shaming you at all! I too am a massive slut and a total cock tease. https://t.co/HbZiZXRi0N
distance: 0.9774 


roberta-base
-------------------

Label: NO
sentence1: Thank you beautiful friend 😊Sending love and 🕯️🚨 light your way 💓 https://t.co/EbPpAKWqjo https://t.co/n3MDADAH7N
--------------------
sentence2: Have a lovely day beautiful sunshine 🌞 ❤️♥️💜🔥🔥🔥🔥🔥🔥🐎 https://t.co/w4yoltPn6z https://t.co/qDf358MMsH
distance: 0.9992 


Label: YES
sentence1: @lkmeenha we can’t even have a day without women making it about themselves 🙄
--------------------
sentence

## Show results

In [ ]:
# COMPLETE

bert-base-uncased

label: NO
sentence1: "I still wish they turned this into a boss fight. https://t.co/HyvPYJPHJc"
--------------------
sentence2: "I don't particularly care or want to know about the cock carousel. Everyone has a past. https://t.co/73WMTyEKHt"
distance: 0.9739


label: YES
sentence1: "The mighty ass. Call me sexist I do not care. https://t.co/LzXw4iRbLR"
--------------------
sentence2: "@RP_JetBlack Not shaming you at all! I too am a massive slut and a total cock tease. https://t.co/HbZiZXRi0N"
distance: 0.9774


roberta-base

label: NO
sentence1: "Thank you beautiful friend 😊Sending love and 🕯️🚨 light your way 💓 https://t.co/EbPpAKWqjo https://t.co/n3MDADAH7N"
--------------------
sentence2: "Have a lovely day beautiful sunshine 🌞 ❤️♥️💜🔥🔥🔥🔥🔥🔥🐎 https://t.co/w4yoltPn6z https://t.co/qDf358MMsH"
distance: 0.9992


label: YES
sentence1: "@lkmeenha we can’t even have a day without women making it about themselves 🙄"
--------------------
sentence2: "@BigDILF01 Can’t go a day